In [2]:
import os
import time
import copy
import multiprocessing as mp


import random
import numpy as np
from collections import namedtuple

from test_env import TestEnv_v2
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
HIDDEN_SIZE = 128
BATCH_SIZE = 64
PERCENTILE = 70
LEARNING_RATE = 0.0005
REUSE_TIMES = 6
GAMMA = 1

INIT_ROUNDS = 10

In [4]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size[1], hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, int(hidden_size/2)),
            nn.ReLU()
        )
        self.out = nn.Linear(obs_size[0] * int(hidden_size/2), n_actions) 

    def forward(self, x):
        x = self.net(x)
        x = x.view(x.size(0), -1)   # to (batch_size, obs_size[0] * hidden_size/2)
        output = self.out(x)
        return output

Episode = namedtuple('Episode', field_names=['reward', 'steps', 'info'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    times = 0
    while True:
        obs_v = torch.FloatTensor([obs]).cuda()
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.cpu().data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, ext_info = env.step(action)
        episode_reward += reward * (GAMMA ** len(episode_steps))
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done or ext_info[3] > 10000:
            batch.append(Episode(reward=episode_reward, steps=episode_steps, info=ext_info))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            # print("round_mean=%.1f, distance_mean=%.1f, steps=%d "% \
            #  (ext_info[1], ext_info[2], ext_info[3]))
            if times%8 == 0:
                print("|", end='')
            print("%d"% (times%8), end='')
            times += 1
            
            if len(batch) == batch_size:
                print(" ")
                times = 0
                yield batch
                del obs_v, act_probs_v, act_probs, ext_info
                batch = []
        obs = next_obs

In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))
    infos = np.array(list(map(lambda s: s.info, batch)))
    info_mean = np.mean(infos,axis=0)

    train_obs = []
    train_act = []
    elite_batch = []
    for example, discounted_reward in zip(batch, rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation, example.steps))
            train_act.extend(map(lambda step: step.action, example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound, reward_mean, info_mean

In [7]:
def get_init_batch(env,batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    r = 0
    test_num = 0
    times = 0
    while True:
        r = random.randint(0,99)
        next_obs, reward, is_done,  ext_info = env.step(r)
        episode_reward += reward * (GAMMA ** len(episode_steps))
        episode_steps.append(EpisodeStep(observation=obs, action=r))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps, info=ext_info))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if times%8 == 0:
                print("|", end='')
            print("%d"% (times%8), end='')
            times += 1
            if len(batch) == batch_size:
                print(" ")
                yield batch
                batch = []
                test_num +=1
                times = 0
                if test_num == INIT_ROUNDS:
                    break
        obs = next_obs

In [8]:
if __name__ == "__main__":
    env = TestEnv_v2()
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    # os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    # os.environ["CUDA_VISIBLE_DEVICES"]="1"
    
    
    # print(device_lib.list_local_devices())
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    obs_size = env.observation_size
    n_actions = env.action_num

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        net = nn.DataParallel(net)

    net = net.to(device)
    
    objective = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(params=net.parameters(), lr=LEARNING_RATE)
    writer = SummaryWriter(comment="-test")

Let's use 4 GPUs!


In [10]:
    optimizer = optim.Adam(params=net.parameters(), lr=0.001)
    
    for batch in get_init_batch(env, 32):
        batch, obs, acts, reward_b, reward_m, info_m = filter_batch(batch, 30)

        obs_v = torch.FloatTensor(obs).cuda()
        acts_v = torch.LongTensor(acts).cuda()

        for i in range(5):
            optimizer.zero_grad()
            action_scores_v = net(obs_v)
            loss_v = objective(action_scores_v, acts_v)
            loss_v.backward()
            optimizer.step()
            print("-", end='')
        print(" ")
            
        print("reward_mean=%.1f, reward_bound=%.1f, round_mean=%.1f, \
        distance_mean=%.1f, steps=%d "% \
          (reward_m, reward_b, info_m[1], info_m[2], info_m[3]))
        del loss_v, acts_v, obs_v, action_scores_v, info_m, reward_m
        torch.cuda.empty_cache()
        
    torch.cuda.empty_cache()
    torch.save(net.state_dict(), 'net_params_init.pkl')  

|01234567|01234567|01234567|01234567 
----- 
reward_mean=-6191.3, reward_bound=-6474.2, round_mean=23.1,     distance_mean=74.4, steps=3077 
|01234567|01234567|01234567|01234567 
----- 
reward_mean=-5721.9, reward_bound=-6015.3, round_mean=23.1,     distance_mean=74.2, steps=2838 
|01234567|01234567|01234567|01234567 
----- 
reward_mean=-5731.6, reward_bound=-6159.8, round_mean=23.0,     distance_mean=74.1, steps=2804 
|01234567|01234567|01234567|01234567 
----- 
reward_mean=-6329.3, reward_bound=-6719.2, round_mean=23.8,     distance_mean=75.2, steps=3119 
|01234567|01234567|01234567|01234567 
----- 
reward_mean=-5914.6, reward_bound=-6297.5, round_mean=22.1,     distance_mean=73.8, steps=2935 
|01234567|01234567|01234567|01234567 
----- 
reward_mean=-5894.9, reward_bound=-6337.9, round_mean=22.9,     distance_mean=74.0, steps=2887 
|01234567|01234567|01234567|01234567 
----- 
reward_mean=-5780.9, reward_bound=-6228.1, round_mean=23.1,     distance_mean=73.5, steps=2942 
|01234567|012

In [ ]:
    writer = SummaryWriter(comment="-test")
    
    # net.load_state_dict(torch.load('net_params_init.pkl'))
    net.load_state_dict(torch.load('net_params.pkl'))
    optimizer = optim.Adam(params=net.parameters(), lr=LEARNING_RATE)
    
    print("start training!!!")
    start = time.time()
    
    res_batch = []
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))
        res_batch, obs, acts, reward_b, reward_m, info_m = filter_batch(res_batch + batch, PERCENTILE)
        if not res_batch:
            continue
            
        end = time.time()
        localtime = time.asctime( time.localtime(time.time()) )
        print("%d: reward_mean=%.1f, reward_bound=%.1f, round_mean=%.1f, \
        distance_mean=%.1f, steps=%d "% \
              (iter_no,  reward_m, reward_b, info_m[1], info_m[2], info_m[3]))
        print("iter time: %d s, localtime:"% (int(end - start)), localtime)
        start = end    
        
        obs_v = torch.FloatTensor(obs).cuda()
        acts_v = torch.LongTensor(acts).cuda()
        res_batch = res_batch[-32:]

        for i in range(REUSE_TIMES):
            optimizer.zero_grad()
            action_scores_v = net(obs_v)
            loss_v = objective(action_scores_v, acts_v)
            loss_v.backward()
            optimizer.step()
            
            print("-", end='')
        print(" ")
        if iter_no%10 == 0:
            torch.save(net.state_dict(), 'net_params.pkl')   
        
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        writer.add_scalar("round_mean", info_m[1], iter_no)
        writer.add_scalar("distance_mean", info_m[2], iter_no)
        writer.add_scalar("step_mean", info_m[3], iter_no)
        # if reward_m > 500:
        #     print("Solved!")
        #     break
        
        del loss_v, acts_v, obs_v, action_scores_v, info_m, reward_m
        torch.cuda.empty_cache()
    writer.close()

start training!!!
|01234567|01234567|01234567|01234567|01234567|01234567|01234567|01234